In [3]:
import torch
from tqdm.notebook import tqdm

In [4]:
def shift(c, size, i):
    res = torch.zeros(size, device=c.device)
    res[:i] = c[i:]
    return res

def toeplitz(c, r):
    if len(c.size()) == 3:
        IC, OC, l_r = r.size()
        l_c = c.size()[-1]
        res = torch.zeros(IC,OC, l_c, l_r, device=c.device)
        # p = torch.zeros(len(c))
        # c = torch.cat([p,c.flip(0)])
        c = c.flip(2)
        c = torch.cat([c,c],dim=2)
        for i in range(1, l_c + 1):
            if i <= l_r:
                res[:,:,i-1,:i] = c[:,:,-i:]
            else:
                res[:,:,i-1,:] = c[:,:,-i:-i+l_r]
        return res
    if len(c.size()) == 1:
        l_r = r.size()[0]
        l_c = c.size()[0]
        res = torch.zeros(l_c, l_r, device=c.device)
        # p = torch.zeros(len(c))
        # c = torch.cat([p,c.flip(0)])
        c = c.flip(0)
        c = torch.cat([c,c])
        for i in range(1, l_c + 1):
            if i <= l_r:
                res[i-1,:i] = c[-i:]
            else:
                res[i-1,:] = c[-i:-i+l_r]
        return res

def vector_to_matrix(input, output_shape):
    B, C, output_h, output_w = output_shape
    output = input.view(B, C, output_h, output_w)
    output=torch.flip(output,dims=[-2])
    return output

def matrix_to_vector(input):
    B, C, input_h, input_w = input.shape
    input = torch.flip(input,dims=[-2]) 
    output_vector = input.view(B,C,-1)
        
    return output_vector


In [5]:
def conv2d(I,F):
    # st = time.time()
    I_B, I_C, I_row_num, I_col_num = I.size() 

    # number of columns and rows of the filter
    F_CO, F_CI, F_row_num, F_col_num = F.size()

    #  calculate the output dimensions
    output_row_num = I_row_num + F_row_num - 1
    output_col_num = I_col_num + F_col_num - 1

    F_zero_padded = torch.zeros(F_CO, F_CI, output_row_num, output_col_num, device=I.device)
    F_zero_padded[:, :, -F_row_num:, :F_col_num] = F
    del(F)

    toeplitz_list = torch.zeros(F_zero_padded.shape[2], F_CO, F_CI, output_col_num ,I_row_num)
    for i in range(F_zero_padded.shape[2]-1, -1, -1): # iterate from last row to the first row
        c = F_zero_padded[:, :, i, :] # i th row of the F 
        r = torch.zeros(F_CO, F_CI, I_col_num, device=I.device)
        r[:,:,0] = c[:,:,0]
        toeplitz_m = toeplitz(c, r)
        toeplitz_list[F_zero_padded.shape[2]-1-i,:,:,:,:] = toeplitz_m

    c = torch.Tensor(range(1, F_zero_padded.shape[2]+1)).to(I.device)
    r = torch.zeros(I_row_num, device=I.device)
    r[0] = c[0]
    doubly_indices = toeplitz(c, r).to(torch.int)

    h = output_col_num * doubly_indices.shape[0]
    w = I_row_num * doubly_indices.shape[1]
    doubly_blocked_shape = [F_CO, F_CI, h, w]
    doubly_blocked = torch.zeros(doubly_blocked_shape, device=I.device)

    for i in range(doubly_indices.shape[0]):
        for j in range(doubly_indices.shape[1]):
            start_i = i * output_col_num
            start_j = j * I_row_num
            end_i = start_i + output_col_num
            end_j = start_j + I_row_num
            doubly_blocked[:,:,start_i: end_i, start_j:end_j] = toeplitz_list[doubly_indices[i,j]-1,:,:,:,:]
    
    # print("sad")
    # print(doubly_blocked.shape)
    vectorized_I = matrix_to_vector(I)
    # print(vectorized_I.shape)
    # print(vectorized_I)
    output_size = output_row_num * output_col_num
    output_vect = torch.zeros(I_B, F_CO, output_size, device=I.device)
    # print(doubly_blocked.shape)
    # for i in tqdm(range(I_B)):
    # for i in range(I_B):
    # print(time.time() - st)
    for k in range(I_C):
        # print(doubly_blocked[:,k].reshape(1,F_CO*h,w).expand(I_B,F_CO*h,w).size())
        output_vect[:,:,:] += torch.bmm(doubly_blocked[:,k].reshape(1,F_CO*h,w).expand(I_B,F_CO*h,w), vectorized_I[:,k,:].view(I_B,I_row_num * I_col_num,1)).view(I_B,F_CO, output_size)
    out_shape = [I_B, F_CO, output_row_num, output_col_num]
    my_output = vector_to_matrix(output_vect, out_shape)
    # print(time.time() - st)
    return my_output

In [4]:
import time
# device = torch.device("cuda:0")
device = torch.device("cpu")
I = torch.randn(32,16,4,4).to(device)
F = torch.randn(64,16,3,3).to(device)
# I = torch.Tensor([[[[1, 2, 3], [4, 5, 6], [7,8,9]], [[1, 2, 3], [4, 5, 6], [7,8,9]]]])
# I = torch.ones(1,2,3,3)
model = torch.nn.Conv2d(16,64,3,padding=2).to(device)
# F = torch.Tensor([[[[10, 20, 30], [40, 50, 60], [70, 80, 90]],[[10, 20, 30], [40, 50, 60], [70, 80, 90]]]])
# F = torch.ones(1,2,3,3)
model.weight.data = F.flip(dims=[-1,-2])
model.bias.data = torch.zeros_like(model.bias)
# print(I.shape)
m_res = model(I)
st = time.time()
a = lol(I,F)#.numpy()
# print(torch.prod(torch.Tensor(a.shape)))
print(time.time() - st)
print(((a - m_res).abs() < 1e-4).sum() / torch.prod(torch.Tensor(list(a.shape))))

0.00799870491027832
0.01299738883972168
tensor(1.)


In [5]:
import numpy as np
# device = torch.device("cpu")
device = torch.device("cuda:0")
total_runs = 100
time_list = []
for _ in tqdm(range(total_runs)):
    I = torch.randn(128,64,16,16).to(device)
    F = torch.randn(64,64,3,3).to(device)
    st = time.time()
    a = lol(I,F)
    time_list.append(time.time()-st)
print(np.mean(time_list))

0.3860034942626953
0.3379991054534912
0.33497095108032227
0.3455071449279785
0.32750844955444336
0.32900023460388184
0.32897496223449707
0.3277566432952881
0.32399940490722656
0.3249998092651367
0.32599878311157227
0.3249692916870117
0.32299137115478516
0.32497525215148926
0.312999963760376
0.31400060653686523
0.3275115489959717
0.31897616386413574
0.31996941566467285
0.324970006942749
0.32900142669677734
0.32297492027282715
0.32697081565856934
0.3319997787475586
0.3260176181793213
0.3249998092651367
0.3359689712524414
0.3640003204345703



KeyboardInterrupt: 

In [54]:
a = torch.randn(32,1, 28,28)
k1 = torch.randn(6,1,5,5)
c1 = conv2d(a,k1)[:,:,2:-2,2:-2]
c1 = torch.nn.functional.max_pool2d(c,2)
k2 = torch.randn(16,6,5,5)
c1 = conv2d(c1,k2)[:,:,2:-2,2:-2]

In [55]:
c1.shape

torch.Size([32, 16, 7, 7])

In [57]:
f = torch.nn.functional.conv2d

c2 = f(a,k1.flip(dims=[-1,-2]),padding=2)
c2 = torch.nn.functional.max_pool2d(c,2)
c2 = f(c2,k2.flip(dims=[-1,-2]),padding=2)

In [59]:
((c1 - c2).abs() < 1e-3).sum() / np.prod([32, 16, 7, 7])

tensor(0.9997)

In [41]:
import numpy as np
np.prod

<function numpy.prod(a, axis=None, dtype=None, out=None, keepdims=<no value>, initial=<no value>, where=<no value>)>